# Tumor Annotations

Programme de ce TP:
- Visualiser des annotations expertes
- Charger un modèle de segmentation automatique
- Appliquer ce modèle sur de nouvelles données
- Générer des cartes de prédictions

![img1](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcontent.iospress.com%2Fmedia%2Fxst%2F2020%2F28-4%2Fxst-28-4-xst200658%2Fxst-28-xst200658-g002.jpg%3Fwidth%3D755&f=1&nofb=1 "Source: https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcontent.iospress.com%2Fmedia%2Fxst%2F2020%2F28-4%2Fxst-28-4-xst200658%2Fxst-28-xst200658-g002.jpg%3Fwidth%3D755&f=1&nofb=1")

Source: https://content.iospress.com/articles/journal-of-x-ray-science-and-technology/xst200658


# Prérequis pour ouvrir le notebook (liste diffusée par mail)

- 0) Créer un compte Google pour avoir accès à Google Drive.
- 1) Ouvrir ce lien de partage : https://drive.google.com/drive/folders/1j-XK2umdsMwAPsOgEgqt4rR56q6oBGSf?usp=sharing
- 2) Dans l'en-tête "Partagés avec moi", aller dans "data", et cliquer sur "ajouter à mon drive".
- 3) Pour ouvrir les TP, aller d'abord dans https://colab.research.google.com/
- 4) Sur la page de chargement (figure ci-dessous): cliquer sur "GitHub", puis rentrer le lien https://github.com/afiliot/TPDUIA ; vous pouvez refuser les demandes d'autorisation. Puis, dans le menu déroulant des fichiers, sélectionnez `TPDUIA/2022/tumor_annotations.ipynb`

# Prérequis après ouverture du notebook

- 5) Cliquer dans le menu sur `Affichage` puis `Dérouler les rubriques`.
- 6) Réaliser un point de montage avec le contenu de son drive pour accéder aux données du TP (cliquez sur la cellule ci-dessous).
- 7) Une fois le notebook ouvert, cliquer sur `Modifier` puis `Paramètres du notebook` puis `accélérateur matériel`, et finalement `GPU` pour disposer d'une carte graphique virtualisée.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

# Données utilisées: programme TCGA

On reprend les mêmes 6 lames qu'au TP précédent, <a href="https://portal.gdc.cancer.gov/image-viewer?filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.case_id%22%2C%22value%22%3A%5B%22set_id%3A_myuY38BEqg9E8t96O1c%22%5D%7D%7D%5D%7D"> **disponible ici**<a>.

(voir le notebook `whole_slide_images` pour plus de détails sur les données TCGA)

# Installer open-slide

Comme précédemment, on exécute la cellule de code ci-dessous pour installer `openslide`.

In [ ]:
!apt update && apt install -y openslide-tools
!pip install openslide-python
!pip install p-tqdm
!git clone https://github.com/openslide/openslide-python.git
!mv openslide-python openslidepython
!wget https://raw.githubusercontent.com/afiliot/TPDUIA/main/TPDUIA/2022/scripts.py
!mv scripts.py /content/gdrive/MyDrive/TPDUIA/

# Ajout du code `scripts.py` dans Colab

In [ ]:
import sys
sys.path.insert(0, '/content/gdrive/My Drive/TPDUIA/')

# Import des librairies nécessaires

In [ ]:
# librairies utilitaires
import os
from pprint import pprint
from typing import Tuple
from glob import glob
from random import sample

# numpy pour les matrics
import numpy as np

# pandas pour les dataframe
import pandas as pd

# visualisation de la progression des processus
from tqdm import tqdm
from p_tqdm import p_map

# manipulation et traitement des WSI
import openslide

# visualisation
import matplotlib
import matplotlib.pyplot as plt

# traitements des images
from scipy.ndimage import gaussian_filter

# Chemin vers les données

(exécutez cette cellule)

In [ ]:
wsi_folder = '/data/freganet/RAW/TCGA/WSI/' #'/content/gdrive/MyDrive/TPDUIA/SLIDES/'
patches_folder = '/data/freganet/TPDU/PATCHES_BACKUP/' #'/content/gdrive/MyDrive/TPDUIA/PATCHES_BACKUP/'

# liste des chemins correspondant aux lames de 0 à 5
wsi_paths = [
    
    '8362/TCGA-BR-8362-01Z-00-DX1.178c994d-2cf5-4385-a807-fc08e0cf10f6.svs', # patient 8362, lame 0
    'A4CS/TCGA-BR-A4CS-01Z-00-DX1.84B5892C-6C7F-474D-BA09-04E870A47764.svs', # patient A4CS, lame 1
    'A91D/TCGA-VQ-A91D-01Z-00-DX1.B7E99494-FA16-42B5-AC1C-1498A7989111.svs', # patient A91D, lame 2
    'A4GQ/TCGA-HU-A4GQ-01Z-00-DX1.699A0E57-FE02-4329-9826-3EFC2DECD408.svs', # patient A4GQ, lame 3
    '8686/TCGA-BR-8686-01Z-00-DX1.8f559dd9-228e-4ed2-a7e4-16e04b80f05c.svs', # patient 8686, lame 4
    '8590/TCGA-BR-8590-01Z-00-DX1.504afa1d-3a8f-423f-9edf-4df35720c05f.svs'  # patient 8590, lame 5
    
]

# Visualisation des lames

On reprend le code pour visualiser une lame.

In [ ]:
# vous pouvez explorer d'autres lames
# 1) exécutez la cellule et entrez un numéro de lame entre 0 et 5
numero_lame = 1
assert numero_lame in range(6), 'Entrez un chiffre entre 0 et 5!'

# chemin de la lame d'intérêt que vous avez sélectionnée
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
print(wsi_path)

Pour lancer un serveur de visualisation sur le port 5000, décommentez et exécutez le code ci-dessus.

In [ ]:
from scripts import launch_server

# lancement d'un serveur sur le port 5000
launch_server(wsi_path)

Puis connectez vous à http://127.0.0.1:5000/ pour visualiser la lame.

Une fois la visualisation terminée, vous pouvez cliquer sur l'icône d'interruption du notebook. La cellule précédente devra afficher "Server stopped" une fois le notebook interrompu.

# Chargement des prédictions du modèle entraîné

In [ ]:
predictions = pd.read_csv('/data/freganet/TPDU/predictions_tcga.csv')
# '/content/gdrive/MyDrive/TPDUIA/TUMOR_MODEL/predictions_tcga.csv')
predictions.head()

In [ ]:
from scripts import wsi_to_numpy

def get_utils(
    wsi: openslide.OpenSlide,
    level: int
) -> Tuple[np.ndarray, int, int, float]:
    """Récupère l'image de la lame à un
    niveau de résolution donnée avec ses
    dimensions et le dézoom associé."""
    # niveau de dezoom correspondant
    # au niveau de résolution indiqué
    # en entrée
    dwn = wsi.level_downsamples[level]
    # image macro correspondante
    image = wsi_to_numpy(wsi, level)
    # taille de l'image macro
    h, w = image.shape[:2]
    print(f'Storing image with shape {h}p x {w}p and dezoom = {dwn:.1f}')
    return image, h, w, dwn

# Visualisation

On commence par charger une lame.

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 0
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')

In [ ]:
def show_heatmap(
    wsi: openslide.OpenSlide,
    predictions: pd.DataFrame,
    patient: str,
    slide: str = 'DX1'
) -> None:
    # on récupère une image macro de cette lame au pénultième niveau de
    # résolution le plus faible
    image, h, w, dwn = get_utils(wsi, wsi.level_count-1)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True)
    # on visualise d'abord l'image brute
    ax = axes[0]
    ax.imshow(image);ax.axis('off')
    ax.set_title(f'Patient {patient}, slide {slide}')
    # puis la carte de chaleur
    # pour cela on initialise un masque avec des valeurs
    # manquantes
    mask = np.ones((h, w))*(np.nan)
    # puis on sélectionne uniquement les patches de la 
    # lame pour lesquels une probabilité d'être tumoral
    # est disponible
    cond = (
        predictions['patient']==patient
    ) & (
        predictions['slide']==slide
    )
    # on récupère les probabilités et les chemins vers les
    # patchs correspondant
    probabilities = predictions.loc[cond, ['tile', 'pTUM']]
    # on itère ensuite sur ces deux listes pour mettre à 
    # jour la carte de chaleur avec les probabilités.
    # attention, il faut tenir compte du dézoom !
    for _, (tile, proba) in probabilities.iterrows():
        # on récupère les coordonnées du patch
        t_s = tile.split('_')
        h_t = int(int(t_s[-5][1:]) / dwn)
        w_t = int(int(t_s[-6][1:]) / dwn)
        dh_t = int(int(t_s[-3][2:]) / dwn)
        dw_t = int(int(t_s[-4][2:]) / dwn)
        # on met à jour le masque
        mask[h_t:(dw_t+h_t), w_t:(dh_t+w_t)] = proba
    # petite manipulation pour lisser le masque de 
    # segmentation avec un filtre gaussien
    idx = np.isnan(mask);mask[idx] = 0
    blur_mask = gaussian_filter(mask, sigma=15, order=0)
    # affichage du masque de probabilités
    ax = axes[1]
    ax.imshow(image);ax.axis('off')
    ax.imshow(blur_mask, cmap='jet', alpha=0.3)
    # ajout d'une échelle de couleur
    cmap = matplotlib.cm.jet
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
    cbar_ax = fig.add_axes([1.00, 0.155, 0.02, 0.6922])
    fig.colorbar(
        matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
        ax=ax,
        cax=cbar_ax,
        label='Probabilité',
        aspect=20
    )
    ax.set_title(f'Carte de chaleur des probabilités ')
    plt.tight_layout()
    plt.show()

In [ ]:
show_heatmap(wsi, predictions, patient, slide)

Vous pouvez changer de lame !

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 1
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')
show_heatmap(wsi, predictions, patient, slide)

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 2
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')
show_heatmap(wsi, predictions, patient, slide)

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 3
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')
show_heatmap(wsi, predictions, patient, slide)

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 4
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')
show_heatmap(wsi, predictions, patient, slide)

In [ ]:
# chemin de la lame d'intérêt que vous avez sélectionnée
numero_lame = 5
wsi_path = os.path.join(wsi_folder, wsi_paths[int(numero_lame)])
wsi = openslide.OpenSlide(wsi_path)
print(f'Slide path: {wsi_path}')
patient = wsi_path.split(os.path.sep)[-2]
slide = 'DX1'
print(f'Patient: {patient}, slide: {slide}')
show_heatmap(wsi, predictions, patient, slide)